# Keras Network to Classify Odd/Even Numbers

In [1]:
import warnings
warnings.filterwarnings('ignore')

# imports
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Model

Using TensorFlow backend.


## Big Picture
- Supervised learning
- Neural networks, like other supervised algorithms, try to approximate any arbitrary function, given enough data

## Goal
- Train a neural network classifier to classify if an integer is odd or even.

## Workflow
1. Setup training data
2. Create binary encoding for the input layer
3. Create binary decoding for the ouput layer
4. Create the model, add layers, and train/fit
5. Create predictions on new data
3. Evaluate the performance of the classifier

In [2]:
# Specify the number of binary digits. 
NUM_DIGITS = 10

In [3]:
## Setup the training data for 101-1023. 1023 is the highest number countable with 10 binary digits
raw_training_data = np.array(range(101, 2**NUM_DIGITS))

In [4]:
raw_training_data[:10]

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110])

In [5]:
## Binary Encoding function
# In order to binary encode the input values
def binary_encode(i, NUM_DIGITS):
    return np.array([i >> d & 1 for d in range(NUM_DIGITS)])

In [6]:
print("number \t [binary digits]")
print(1, "\t", binary_encode(1, NUM_DIGITS))
print(2, "\t", binary_encode(2, NUM_DIGITS))
print(3, "\t", binary_encode(3, NUM_DIGITS))
print(4, "\t", binary_encode(4, NUM_DIGITS))
print(4, "\t", binary_encode(5, NUM_DIGITS))

number 	 [binary digits]
1 	 [1 0 0 0 0 0 0 0 0 0]
2 	 [0 1 0 0 0 0 0 0 0 0]
3 	 [1 1 0 0 0 0 0 0 0 0]
4 	 [0 0 1 0 0 0 0 0 0 0]
4 	 [1 0 1 0 0 0 0 0 0 0]


In [7]:
# encode the training data
encoded_training_data = [binary_encode(i, NUM_DIGITS) for i in raw_training_data]

In [8]:
x_train = np.array(encoded_training_data)

In [9]:
x_train

array([[1, 0, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [10]:
# Encode "even" and "odd" to zero or 1
# If we had more than 2 options, we'd one hot encode to make an array
def encode_even_odd(i):
    if i % 2 == 0:
        return [0]
    else:
        return [1]

assert encode_even_odd(2) == [0]
assert encode_even_odd(3) == [1]

In [11]:
# y_train is the encoded output. 
# This is our "labeled data" for supervised learning
y_train = np.array([encode_even_odd(i) for i in range(101, 2 ** NUM_DIGITS)])

In [12]:
print("Number representeed as a decimal", raw_training_data[0])
print("Number represented as binary in a list:", x_train[0])
print("0 for even, 1 for odd:", y_train[0])

Number representeed as a decimal 101
Number represented as binary in a list: [1 0 1 0 0 1 1 0 0 0]
0 for even, 1 for odd: [1]


## Compare raw inputs to the lables
- The first five values in the input tensor are 101, 102, 103, 104, and 105
- The encoded labels specify "odd", "even", "odd", "even", and then "odd"
- Remember that training a classifier is supervised learning with labeled data

In [30]:
print("Raw training data", raw_training_data[:5])
print(y_train[:5].flatten())

Raw training data [101 102 103 104 105]
[1 0 1 0 1]


In [14]:
# Convert from binary back to the ouput string
def decode_even_odd(n):
    if n == 0:
        return "even"
    else:
        return "odd"

assert decode_even_odd(0) == 'even'
assert decode_even_odd(1) == 'odd'

In [15]:
# Another way I've seen this decoding written (especially with one hot encoded options)
# Convert from binary back to the ouput string
def decode_even_odd2(n):
    return ["even", "odd"][n]

assert decode_even_odd2(0) == 'even'
assert decode_even_odd2(1) == 'odd'

In [16]:
# Now let's build our model, add layers, compile, and fit it!
model = Sequential()

## Add the model's layers.
# These are defaults and they work pretty well
model.add(Dense(1000, input_dim=NUM_DIGITS, activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(4, activation="softmax"))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adagrad', metrics=["accuracy"])

In [32]:
# Fit the model on the training data
model.fit(x_train, y_train, nb_epoch=3, batch_size=128)

Epoch 1/3
923/923 [==============================] - 0s 171us/step - loss: 4.5523e-06 - acc: 1.0000
Epoch 2/3
923/923 [==============================] - 0s 180us/step - loss: 4.4970e-06 - acc: 1.0000
Epoch 3/3
923/923 [==============================] - 0s 174us/step - loss: 4.4490e-06 - acc: 1.0000


In [18]:
# Setup x_test for numbers from 1 to 100 (the model hasn't seen these at all)
numbers = np.arange(1, 101)
x_test = np.transpose(binary_encode(numbers, NUM_DIGITS))
y_test = model.predict_classes(x_test)

In [19]:
# Setup predicted output
predictions = np.vectorize(decode_even_odd)(y_test)
print (predictions)

['odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even'
 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even'
 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even'
 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even'
 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even']


In [20]:
actual = np.array([])
for i in numbers:
    if i % 2 == 0:
        actual = np.append(actual, "even")
    else:
        actual = np.append(actual, "odd")

actual

array(['odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even'], dtype='<U32')

In [21]:
# Let's evaluate the model's predictions
evaluate = np.array(actual == predictions)
print (np.count_nonzero(evaluate == True) / 100)

1.0


### Ok, 100% accuracy... but what if we send the model random numbers?

In [22]:
random_numbers = np.random.randint(1023, size=10000)
random_numbers[:10]

array([ 501,  847,  442,  828,  559,  459,  373, 1022,  215,  942])

In [23]:
x_test = np.transpose(binary_encode(random_numbers, NUM_DIGITS))
y_test = model.predict_classes(x_test)

In [24]:
predictions = np.vectorize(decode_even_odd)(y_test)

In [25]:
predictions.shape

(10000,)

In [26]:
actual = np.array([])
for i in random_numbers:
    if i % 2 == 0:
        actual = np.append(actual, "even")
    else:
        actual = np.append(actual, "odd")

actual

array(['odd', 'odd', 'even', ..., 'even', 'odd', 'odd'], dtype='<U32')

In [27]:
actual.shape

(10000,)

In [28]:
# Let's evaluate the model's predictions
evaluate = np.array(actual == predictions)
print (np.count_nonzero(evaluate == True) / 100)

100.0


In [29]:
# make predictions on 1-100 
# make predictions on 1024-2048
